In [1]:
import pandas as pd
import numpy as np
import vnstock as vn

In [2]:
# Load data
START = "2019-01-01"
END = "2024-02-01"
def load_data(ticker):
        data = vn.stock_historical_data(ticker, START, END)    
        return data
data = load_data('VCB')
# data = data.set_index('time')
data

,time,open,high,low,close,volume,ticker
0,2019-01-02,34750,35010,34490,34560,1081640,VCB
1,2019-01-03,34820,34880,33980,34560,1071350,VCB
2,2019-01-04,34370,35080,33910,35080,1307310,VCB
3,2019-01-07,35660,35780,35140,35200,1175810,VCB
4,2019-01-08,35460,35590,35140,35460,1318810,VCB
...,...,...,...,...,...,...,...
1269,2024-01-26,91600,92000,91400,92000,953100,VCB
1270,2024-01-29,91800,92000,90900,91200,730500,VCB
1271,2024-01-30,91200,91200,90100,91000,888600,VCB
1272,2024-01-31,90600,90600,88500,88500,945600,VCB


# Volume

In [3]:
mean_20 = data['volume'].rolling(window=20).mean() #Tính trung bình 20 phiên gần nhất
data['on-balance_volume'] = data['volume']/mean_20 #Tính on-balance volume
data

,time,open,high,low,close,volume,ticker,on-balance_volume
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN
...,...,...,...,...,...,...,...,...
1269,2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849
1270,2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792
1271,2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869
1272,2024-01-31,90600,90600,88500,88500,945600,VCB,0.690458


In [4]:
def label_values(a):
    if pd.isnull(a):
        return a
    elif a < 0.75:
        return 'low'
    elif 0.75 <= a <= 1.25:
        return 'medium'
    else:
        return 'high'
        
data['OBV_label'] = data['on-balance_volume'].apply(label_values)
data.sample(10)

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label
542,2021-03-08,63610,63610,62300,62430,1501800,VCB,1.064382,medium
30,2019-02-20,38230,38430,37980,38110,818480,VCB,0.832728,medium
175,2019-09-18,52290,53190,51900,51970,809820,VCB,1.184904,medium
1223,2023-11-22,86300,86700,85500,86700,1012700,VCB,0.814683,medium
538,2021-03-02,65580,66760,64269,65120,1189600,VCB,0.835892,medium
199,2019-10-22,54290,55320,54290,55060,473090,VCB,0.542746,low
1165,2023-08-30,88500,89500,88500,89100,1350000,VCB,1.019079,medium
1064,2023-04-06,77480,77980,77220,77390,350000,VCB,0.463076,low
167,2019-09-06,50290,50290,49650,49840,287990,VCB,0.460328,low
930,2022-09-23,65370,65790,64180,64269,1034100,VCB,0.904685,medium


# Bar

In [5]:
def compare_close_prices(row):
        t = row['close']
        t_minus_1 = row['close_t_minus_1']
        if t > t_minus_1:
            return 'up-bar'
        else:
            return 'down-bar'

data['close_t_minus_1'] = data['close'].shift(1) #Giá đóng cửa ngày t-1

data['bar_type'] = data.apply(compare_close_prices, axis=1)
data.sample(10)

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type
668,2021-09-07,65489,66080,65420,65550,1041800,VCB,0.744204,low,65550.0,down-bar
194,2019-10-15,54800,55060,54610,54930,648400,VCB,0.635734,low,54800.0,up-bar
919,2022-09-08,68500,68500,66050,66050,1074300,VCB,0.927452,medium,67740.0,down-bar
544,2021-03-10,62300,62830,61650,62630,1063900,VCB,0.757001,medium,62300.0,up-bar
754,2022-01-06,66550,67650,65960,66980,1215600,VCB,1.037007,medium,66050.0,up-bar
99,2019-06-03,42880,43330,41260,42880,593660,VCB,0.992609,medium,43330.0,down-bar
90,2019-05-21,43780,44100,43650,43650,473170,VCB,0.792381,medium,43710.0,down-bar
1215,2023-11-10,87100,87500,86000,86000,1352000,VCB,1.386759,high,87900.0,down-bar
1011,2023-01-16,72820,74940,72820,73920,735200,VCB,0.634088,low,72650.0,up-bar
812,2022-04-05,70110,70450,69940,70280,526100,VCB,0.514005,low,69940.0,up-bar


# Spread

In [6]:
data['para'] =abs(data['close'] - data['open'])/(data['high'] - data['low'])
data

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1269,2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667
1270,2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455
1271,2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818
1272,2024-01-31,90600,90600,88500,88500,945600,VCB,0.690458,low,91000.0,down-bar,1.000000


In [7]:
def label_spread(a):
    if 0 < a < 0.3: return 'low'
    elif 0.3 <= a <= 0.7: return 'medium'
    else: return 'high'

data['label_spread'] = data['para'].apply(label_spread)
data

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667,medium
1270,2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455,medium
1271,2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818,low
1272,2024-01-31,90600,90600,88500,88500,945600,VCB,0.690458,low,91000.0,down-bar,1.000000,high


## Closing bar

In [8]:
def label_close_bar(data):
    q1 = data['low'] + (data['high'] - data['low'])/3
    q2 = data['low'] + 2*((data['high'] - data['low'])/3)
    if data['close'] < q1:
        return 'bottom-third'
    elif q1 <= data['close'] <= q2:
        return 'middle-third'
    else:
        return 'top-third' 

data['close_bar_label'] = data.apply(label_close_bar, axis=1)
data

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667,medium,top-third
1270,2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455,medium,bottom-third
1271,2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818,low,top-third
1272,2024-01-31,90600,90600,88500,88500,945600,VCB,0.690458,low,91000.0,down-bar,1.000000,high,bottom-third


In [9]:
data['close_bar_label'].value_counts()

close_bar_label
top-third       492
bottom-third    428
middle-third    354
Name: count, dtype: int64

In [10]:
data.head()

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third
